# midiAI

In this notebook, I'll go through my process of using `music21` to create lists of information about songs generated from the `.midi` files in `data/` generated from the [*OpenBook*](https://veltzer.github.io/openbook/) repository.

Let's begin by importing our libraries. We're going to be using `midi2audio` to playback our generated `.wav`.

In [199]:
from IPython.display import Audio
from midi2audio import FluidSynth
from music21 import converter, harmony, pitch, chord, key, interval, note, stream
from pathlib import Path
from pprint import pprint

import subprocess

## `.midi` file playback

Let's get our playback system running by defining our `FluidSynth` using the `gm-soundfont.sf2` from the project.

In [200]:
fs = FluidSynth(sound_font="assets/gm-soundfont.sf2")

Let's make it easier to create `.wav` files for playback with this quick `.midi` to `.wav` function that creates the `.wav` file and `return`s the output filename.

We're also going to need to hide the wildly long fluidsynth warning outputs, so we'll do that here.

In [201]:
def play_midi(mid_path):
	output_dir = Path("out")
	output_dir.mkdir(exist_ok=True)
	output_name = output_dir / Path(mid_path).with_suffix(".wav").name
	
	# fs.midi_to_audio(mid_path, output_name)
	subprocess.run(
		["fluidsynth", "-ni", "gm-soundfont.sf2", str(mid_path), "-F", str(output_name), "-r 22050", "-q"],
		stdout=subprocess.DEVNULL,
		stderr=subprocess.DEVNULL
	)

	return output_name

As a test, we're going to work with `solar.midi` from `data/`.

In [202]:
solar_path = "data/solar.midi"

Now we can give our `play_midi` function a go and make sure it's working. If you're following along with this notebook, make sure `fluidsynth` is installed using `brew install fluidsynth` if on a Mac.

In [203]:
# Audio(play_midi(solar_path))

## `.music21` parsing

Let's get into using `music21` to sort through our `.midi` file.

In [204]:
solar = converter.parse(solar_path)

We'll inevitably come into problems trying to analyze the `harmony.chordSymbolFromChord` when `music21` can't figure out what the chord symbol should be because there are sharp symbols instead of flat symbols, so let's prepare a couple of `dict`s to replace those notes if we come across them.

In [205]:
note_replacements = {
	"A#": "B-", "C#": "D-", "D#": "E-", "F#": "G-", "G#": "A-"
}

note_replacements_reverse = {
	"B-": "A#", "D-": "C#", "E-": "D#", "G-": "F#", "A-": "G#"
}

### `chord_measures`

Now let's get into creating lists of chord names and note information from each measure. The chords will always be in the first part of the parsed `.midi` file (index 0) and the notes will always be the second part (index 1).

We'll then create empty lists that will hold each measure of chord(s) and note information. You'll notice I had to replace a few sharp notes with flats in order to get the correct chords as well as completely replacing the chord name for a G7b9, which was showing up as G7addG# (essentially the same thing).

### `note_measures`

The note information will be stored in the `note_measures` list where each note is a list with the note's *name* and *octave* in one string and the length of the note (in terms of `music21`'s `quarterNote`) in the next element of the list.

In [206]:
chords = solar.parts[0]
melody = solar.parts[1]

chord_measures = []
for item in chords:
	measure = []
	for c in item:
		if "chord.Chord" in str(c):
			cs = harmony.chordSymbolFromChord(c)
			if "Cannot" in cs.figure:
				pitches = [p for p in c.pitches]
				for idx, p in enumerate(pitches):
					if p.accidental is not None and "#" in p.name:
						pitches[idx] = pitch.Pitch(note_replacements[p.name], octave=p.octave)
				new_cs = harmony.chordSymbolFromChord(chord.Chord(pitches))
				measure.append(new_cs.figure)
			else:
				if f"add{cs.figure[0]}#" in cs.figure:
					nc = cs.figure.replace(f"add{cs.figure[0]}#", "b9")
					measure.append(nc)
				else:
					measure.append(cs.figure)
	chord_measures.append(measure)

note_measures = []
for item in melody:
	measure = []
	for n in item:
		if "note.Note" in str(n) or "note.Rest" in str(n):
			n_name = n.nameWithOctave if "note.Note" in str(n) else "RS"
			n_len = n.quarterLength
			measure.append([n_name, n_len])
	note_measures.append(measure)

Let's take a look at measure one of `chord_measures` and `note_measures`.

In [207]:
print(chord_measures[0])
print(note_measures[0])

['Cm']
[['RS', 0.5], ['C5', 1.5], ['B4', 1.0], ['D5', 0.5], ['C5', 0.5]]


## Sidestep, transposing

Let's say we have a song in the key of Eb major. For training purposes, I want to only train and generate songs in C major. Can we transpose using `music21`?

In [208]:
misty_path = "data/misty.midi"

In [209]:
# Audio(play_midi(misty_path))

In [210]:
misty = converter.parse(misty_path)

Below, I show the route I went with to convert a `.midi` file to C major/A minor. For this purpose, I also exported it as a `.mid` file for playback so you can compare with the previous one.

In [211]:
misty_key = misty.flat.getElementsByClass(key.KeySignature)[0]
target_key = key.Key("C") if misty_key.mode == "major" else key.Key("A", "minor")
i = interval.Interval(misty_key.tonic, target_key.tonic)

misty_in_c = misty.transpose(i)
# misty_in_c.write("midi", fp="misty_in_c.mid")
# Audio(play_midi("misty_in_c.mid"))

/Users/christopherwright/GitHub Projects/midi-ai/.venv/lib/python3.13/site-packages/music21/stream/base.py:3675: Music21DeprecationWarning: .flat is deprecated.  Call .flatten() instead
  return self.iter().getElementsByClass(classFilterList)


So now that we know how to make sure we have consistent data for training, we can get back to where we were.

## Loops -> functions

Let's make functions out of our loops from before, as well as whatever else we can, to make things easier in the future.

In [212]:
def gen_chords(chords_part):
	chord_measures = []
	for item in chords_part:
		measure = []
		for c in item:
			if "chord.Chord" in str(c):
				c_name = ",".join([cn.name for cn in c.pitches])
				c_len = str(c.quarterLength)
				measure.append([c_name, c_len])
		chord_measures.append(measure)
	return chord_measures

def gen_notes(notes_part):
	note_measures = []
	for item in notes_part:
		measure = []
		for n in item:
			if "note.Note" in str(n) or "note.Rest" in str(n):
				n_name = n.nameWithOctave if "note.Note" in str(n) else "RS"
				n_len = str(n.quarterLength)
				measure.append([n_name, n_len])
		note_measures.append(measure)
	return note_measures

def parse_midi_in_c(file_path):
	song = converter.parse(file_path)
	original_key = song.flatten().getElementsByClass(key.KeySignature)[0]
	if original_key.tonic == "C" and original_key.mode == "major":
		return song
	target_key = key.Key("C") if original_key.mode == "major" else key.Key("A", "minor")
	i = interval.Interval(original_key.tonic, target_key.tonic)

	new_song = song.transpose(i)
	return new_song

def gen_data(file_path):
	song = parse_midi_in_c(file_path)
	chords = gen_chords(song.parts[0])
	notes = gen_notes(song.parts[1])
	return [chords, notes]

In [213]:
strange_fruit = gen_data("data/strange_fruit.midi")
print(strange_fruit[1][0])

[['A4', '1.0'], ['A4', '1.0'], ['A4', '1.0'], ['A4', '0.5'], ['A4', '0.5']]


I'm going to now import `tqdm` to show our loading process for importing songs.

In [214]:
from tqdm.notebook import tqdm

In [215]:
data_path = Path("data")
files = [f for f in data_path.iterdir() if f.is_file() and f.suffix.lower() == ".midi"]

# all_songs = [
#   # gen_data(fp) for fp in tqdm(files, unit="file")
#   [gen_data(fp) for fp in tqdm(files, unit="file")]
# ]

all_songs = []

for fp in tqdm(files, unit="file"):
  chords, notes = gen_data(fp)
  song = [chords, notes]
  all_songs.append(song)

  0%|          | 0/153 [00:00<?, ?file/s]

## Tokenizing our data

So now we have a list `all_songs` where the first index, `all_songs[0]`, `all_songs[1]`, etc. are the different songs and the second index, `all_songs[0][0]`, `all_songs[0][1]`, are the chord info and note info respectively.

Now is the tricky part, what is going to be the best way to tokenize this data?

First, what I'll need to do is figure out a good way to get my data into some easy-to-understand information for training. I'd really like to train my model on basically *each measure* to get my note probabilities *while being dependent on what the underlying chord is playing behind those notes* so that my model will be able to **generate melodies based on what chord is input**.

Looks like I'll have to do some more reading and studying to figure out how to tackle this but I'm really happy with where I am at this point. I've taken my `.midi` files and parsed the notes and chords for each measure and created a list `all_songs` that holds every song in my `data/` directory.